# Gramáticas basadas en rasgos

Para esta clase, puede ser de utilidad la siguiente bibliografía:

- [NLTK Book: Chapter 9, Building Feature Based Grammars](https://www.nltk.org/book/ch09.html)
- [NLTK (documentación de librería): Sample usage for featstruct](https://www.nltk.org/howto/featstruct.html)

In [1]:
#Temas de Python que me parece que vamos a tocar en esta clase:
#
#- Diccionarios
#- Conjuntos
#- Funciones
#- Herencia
#- `*args`
#- Métodos de NLTK
#- Función `lambda`

In [2]:
# importa librería nltk
import nltk

In [3]:
# comentario
# no sé si al inicio porque me parece medio colgado, pero podríamos hacer esto que vos querías de ir a la carpeta
# donde está ubicada la librería y mostrarles que no es otra cosa que una carpeta con archivos y código en Python
# no sé si conviene hacerlo en esta instancia o más adelante, cuando importemos el FeatStruct, por ejemplo
# PD: voy a ir dejando comentarios así
#
# IMPORTANTE
# Ju, en esta notebook armé estructuras con latex, pero cuando transformo la notebook a md no
# se ven bien, parace que hay un tema con el conversor y también que GitHub no las renderiza bien
# probé conviritiendo a html y sí se ven bien (al menos en el html, después veremos en la página).
# No sé cómo le meteríamos los botones y el menú navegable a ese html ahora que lo pienso.
# Hay que seguir iterando capaz.

## Estructuras de rasgos

In [4]:
# pongo algunas estructuras para que nos sirvan
# de guía para ir armando con código
# elegí estas porque
# - la de torta nos permite mostrar el rasgo PLU como un bool
# - las otras nos permiten mostrar cómo armar rasgos complejos
# - el ejemplo de la unificación (es el mismo que estaba en clase)
# nos va a permitir pensar el tema de la implementación de la
# unificación (podemos representar la estructura como un diccionario
# pero después no tenemos cómo unificar eso a menos que lo
# implementemos o que ya alguien haya hecho algo similar por
# nosotros, en este caso, les muchaches de NLTK)

\begin{equation}
N = \begin{bmatrix}
\text{SIGNIFICANTE torta}\\
\text{LEXEMA torta}\\
\text{CAT N}\\
\text{GEN fem}\\
\text{PLU -}\\
\end{bmatrix}
\end{equation}

\begin{equation}
N = \begin{bmatrix}
\text{LEX virus}\\
\text{CAT N}\\
\text{CONC }\begin{bmatrix}
\text{NUM [ ]}\\
\text{GEN masc}
\end{bmatrix}
\end{bmatrix}
\end{equation}

\begin{equation}
ADJ = \begin{bmatrix}
\text{LEX mutantes}\\
\text{CAT ADJ}\\
\text{CONC }\begin{bmatrix}
\text{NUM pl}\\
\text{GEN masc}
\end{bmatrix}
\end{bmatrix}
\end{equation}

### Representación con diccionarios

Los diccionarios son un tipo de objeto primitivo de Python en el que podemos agregar entradas, llamadas llaves (_keys_), y asignarles un valor o dato asociado (_value_). Un dato importante para la construcción de diccionarios es que estos no pueden tener _keys_ repetidas. En caso de querer insertar una _key_ que ya se encontraba en un diccionario, se sobreescribirá su valor.

Podemos aprovechar este objeto para representar una estructura de rasgos.

In [5]:
fs_a_dict = {
    'SIGNIFICANTE':'torta',
    'LEXEMA':'torta',
    'CAT':'N',
    'GEN':'fem',
    'PLU':False
}
fs_a_dict

{'SIGNIFICANTE': 'torta',
 'LEXEMA': 'torta',
 'CAT': 'N',
 'GEN': 'fem',
 'PLU': False}

In [6]:
fs_a_dict['GEN']

'fem'

In [7]:
# sobreescribe un valor

fs_a_dict['LEXEMA']='tortas'
fs_a_dict['PLU']=True

fs_a_dict

{'SIGNIFICANTE': 'torta',
 'LEXEMA': 'tortas',
 'CAT': 'N',
 'GEN': 'fem',
 'PLU': True}

`True` y `False` son objetos de tipo booleano, permiten indicar que algo es verdadero o falso, respectivamente. Aquí los usamos para indicar si el atributo _PLU_ está presente o no. También podríamos usar un `string` que indicase "+" o "-", pero este tipo de dato resulta más apropiado para lo que se desea representa.

In [8]:
# arma un diccionario vacío

fs_b_dict = dict()
fs_b_dict

{}

In [9]:
# agrega rasgos de a uno

fs_b_dict['LEX'] = 'virus'
fs_b_dict

{'LEX': 'virus'}

In [10]:
fs_b_dict['CAT'] = 'N'
fs_b_dict

{'LEX': 'virus', 'CAT': 'N'}

In [11]:
# agrega un rasgo complejo

fs_b_dict['CONC'] = {
    'NUM':None,
    'GEN':'masc'
}

fs_b_dict

{'LEX': 'virus', 'CAT': 'N', 'CONC': {'NUM': None, 'GEN': 'masc'}}

`None` es un valor que se utiliza para indicar "nulo". Aquí lo usamos para indiar que el atributo _NUM_ no se encuentra especificaco.

In [12]:
# arma un diccionario vacío

fs_c_dict = dict()
fs_c_dict

{}

In [13]:
# actualiza el diccionario vacío
# con el contenido de otro diccionario

fs_c_dict.update(
    {
        'LEX': 'mutantes',
        'CAT': 'ADJ',
        'CONC': {
            'NUM': 'pl',
            'GEN': 'masc'}
    }
)

fs_c_dict

{'LEX': 'mutantes', 'CAT': 'ADJ', 'CONC': {'NUM': 'pl', 'GEN': 'masc'}}

¿Podríamos utilizar el método `update` para realizar la unificación de dos estructuras así representadas? Probémoslo.

In [14]:
fs_sn_dict = dict()
fs_sn_dict.update(fs_b_dict)
fs_sn_dict

{'LEX': 'virus', 'CAT': 'N', 'CONC': {'NUM': None, 'GEN': 'masc'}}

In [15]:
fs_sn_dict.update(fs_c_dict)
fs_sn_dict

{'LEX': 'mutantes', 'CAT': 'ADJ', 'CONC': {'NUM': 'pl', 'GEN': 'masc'}}

El atributo _CONC_, cuyo atributo _NUM_ se encontraba subespecificado, fue reemplazado con la información más específica de la estructura del ítem léxico "mutantes". Sin embargo, este procedimiento también sobreescribió el valor del atributo _CAT_ y le asignó una categoría que no es la quisiéamos ver en ese rasgo.

Probemos qué sucede si lo hacemos de manera inversa, primero actualizando con los valores del adjetivo y, luego, del nombre:

In [16]:
fs_sn_dict = dict()
fs_sn_dict.update(fs_c_dict)
fs_sn_dict

{'LEX': 'mutantes', 'CAT': 'ADJ', 'CONC': {'NUM': 'pl', 'GEN': 'masc'}}

In [17]:
fs_sn_dict.update(fs_b_dict)
fs_sn_dict

{'LEX': 'virus', 'CAT': 'N', 'CONC': {'NUM': None, 'GEN': 'masc'}}

Ahora tenemos una categoría que se aproxima un poco más a lo que quisiéramos (sin ser el "SN" que nos gustaría), pero tenemos subespecificado el atributo _NUM_.

El método `update`, como recurso para unificar estrucuras, solo resulta útil (y no conlleva efectos indeseados) cuando las estructuras involucradas contienen exactamente los mismo valores asignados a los mismos atributos o tienen rasgos no compartidos (_i.e._ alguna o ambas tiene un par <atributo, valor> que la otra no). Recordemos las estructuras indicadas por Blevins(2011) para "él" (D, izq.) y "canta" (D, der.).

\begin{equation}
\begin{bmatrix}
\text{PER 3}\\
\text{NUM sg}\\
\text{GEN masc}\\
\text{CASO nom}
\end{bmatrix}
\sqcup
\begin{bmatrix}
\text{PER 3}\\
\text{NUM sg}\\
\text{CASO nom}
\end{bmatrix}
\end{equation}

In [18]:
fs_d_1 = {
    'PER':3,
    'NUM':'sg',
    'GEN':'masc',
    'CASO':'nom'
}
fs_d_2 = {
    'PER':3,
    'NUM':'sg',
    'CASO':'nom'
}

In [19]:
fs_d_1

{'PER': 3, 'NUM': 'sg', 'GEN': 'masc', 'CASO': 'nom'}

In [20]:
fs_d_2

{'PER': 3, 'NUM': 'sg', 'CASO': 'nom'}

In [21]:
fs_d = fs_d_1
fs_d.update(fs_d_2)
fs_d

{'PER': 3, 'NUM': 'sg', 'GEN': 'masc', 'CASO': 'nom'}

¿Y cómo podríamos evaluar la subsunción?

La comparación de objetos por igualdad nos permite ver si ambos objetos son exatamente iguales, pero no si uno de ellos se encuentra contenido en el otro.

In [40]:
fs_d_1 == fs_d_1

True

In [42]:
fs_d_2 == fs_d

False

Probemos la siguiente función, implementada para comparar la subsunción entre dos estructuras representadas en un diccionario:

In [35]:
def subsumes_dict(general, specific):
    # atributos de esturctura general
    general_attr = set(general.keys())
    # atributos de esturctura específica
    specific_attr = set(specific.keys())
    # chequea que los atributos de e.general son un subconjunto de e.específica
    if general_attr.issubset(specific_attr):
        # chequea que los valores de esos atributos sean iguales
        # matches es una lista con valores booleanos
        # True si los valores son iguales y False si son distintos
        matches = [general[attr] == specific[attr] for attr in general_attr]
        # all() devuelve True si todos los booleanos de la lista son True
        # y False si alguno es False
        return all(matches)
    # si no, devuelve Falso
    # (hay atributos en e.general que no están en e.específica
    else:
        return False

In [43]:
subsumes_dict(fs_d, fs_d_2)

False

In [44]:
subsumes_dict(fs_d_2, fs_d)

True

Limitaciones de la representación con diccionarios:

- no brinda un método sencillo para realizar el proceso de unificación
- no brinda un método sencillo para realizar el proceso de subsumción

### Representación como conjuntos

In [46]:
{tuple(['attribute','value'])}

{('attribute', 'value')}

### Representación con NLTK

In [ ]:
fs1 = nltk.FeatStruct(TENSE='past', CHICHO=True)

In [ ]:
fs2 = nltk.FeatStruct(CAT='V')

In [ ]:
fs3 = nltk.FeatStruct(CAT='V', TENSE='present')

In [ ]:
fs3 = nltk.FeatStruct("""[CAT=V, TENSE=present]""")

In [ ]:
# explicar representación de inespecificidad

### Representación con un grafo

In [ ]:
# armar

## Estructuras de rasgos como funciones

Acá podemos mostrar:
- función que toma un atributo y devuelve un valor
- función que toma un path de atributos y devuelve el valor siguiendo ese path

Para esto -> armar una clase que herede de FeatStruct e implementar un método que recorra la estructura. Como argumento de la función usar `*args`.

Podemos aprovechar y ver el paralelismo con la noción de herencia, que tiene una procedencia computacional.

## Unificación

In [ ]:
print(fs1.unify(fs3, trace=1))

Acá mostrar:

- unificación exitosa
- unificación fallida
- unificación vs. unión: antes vimos que las estructuras se pueden representar como conjuntos. Este es un buen momento para mostrar que la unificación y la unión funcionan distinto y que la única situación en la que funcionan igual es con rasgos atómicos.

## Subsunción

In [ ]:
fs2.subsumes(fs3)

## Rasgos atómicos y complejos

In [ ]:
fs = nltk.FeatStruct(CAT='V', NUM='plu')

In [ ]:
fs

In [ ]:
fs['CONCORD'] = fs

In [ ]:
fs

In [ ]:
fs.cyclic()

In [ ]:
# retomar feature sharing
# recordar que compartir rasgos no es lo mismo que tener duplicada la misma información
# mostrar con equal_values y hablar sobre reentrancia

In [ ]:
fs.equal_values(fs, check_reentrance=False)

Acá también podríamos mostrar que una estructura que comparte rasgos es más específica que una que no los comparte aunque tiene esepecificados los mismos valores para los mismos atributos.

## Construcción de una gramática

In [ ]:
nltk.data.show_cfg('gramaticas/GramaticaDeRasgos.fcfg')

In [ ]:
sentence = 'los chicas caminan'
tokens = sentence.split()
print(sentence)
print(type(sentence))
print(tokens)
print(type(tokens))

In [ ]:
from nltk import load_parser
cp = load_parser('gramaticas/GramaticaDeRasgos.fcfg', trace=2)
for tree in cp.parse(tokens):
     print(tree)

## Uso de rasgos para significado

Los rasgos pueden utilizarse a su vez para construir una representación semántica de las oraciones.

En semántica formal existe una función particular, que se conoce con el nombre de función interpretación, y que se anota con corchetes dobles. La función interpretación devuelve por cada expresión lingüística su denotación. Las denotaciones pueden ser de dos tipos: 

- elementos atómicos (típicamente objetos o proposiciones, pero también hay otras ontologías que incluyen mundos posibles, eventos y tiempos, entre otras cosas)
- funciones. 

El uso de rasgos para dar cuenta del significado consiste en que la función denotación sea el valor de un rasgo semántico.

En el [libro de NLTK](https://www.nltk.org/book/ch10.html) y en la [documentación de NLTK](http://nltk.sourceforge.net/doc/en/ch11.html) hay información sobre cómo implementar esto mediante una gramática de rasgos en NLTK.


Las funciones equivalen a conjuntos y se expresan en el llamado cálculo lambda. 

\x. x fuma

Esta es una función que toma un x y devuelve verdadero si x fuma y falso si x no fuma. En términos de conjuntos equivale al conjunto de todos los fumadores (ténicamente equivale al conjunto característico de todos los fumadores, que es el que devuelve verdadero si x fuma y falso si x no fuma).

Hay dos operaciones básicas de cálculo lambda que son particularmente relevantes (una tercera no tuvo tanta repercusión en la semántica formal):

- Conversión alpha (o reducción alpha): cambiar el nombre de una variable y, conjuntamente, el de todas las variables ligadas con ella.
`[\x. x fuma] = [\y. y fuma] = [\z. z fuma] = ...`
- Conversión lambda (o reducción beta): cuando combinamos una función con un argumento, eliminar el prefijo lambda y reemplazar todas las ocurrencias de la variable que introduce ese prefijo por el argumento.

- `[\x. x fuma](cata) = cata fuma`
- `[\f. f](\x. x fuma) = \x. x fuma`
- `[\f. [\g. [\x. g(x)=f(x)=1]]](\x. x fuma)(\x. x baila) = [\x. [\x. x fuma](x)=[\x. x baila](x)=1] = [\x x fuma y x baila]` 

Las interpretación semántica de las expresiones lingüísticas se da a partir del significado de sus partes y su combinación mediante reglas que dependen de la forma del árbol y de los tipos de las funciones. Las reglas más frecuentes son: 

- Aplicación funcional: Si un nodo A domina a dos nodos B y C tales que B es una función cuyo dominio incluye a C, entonces [[A]]=[[B]]([[C]])
- Modificación de predicados: Si un nodo A domina a dos nodos B y C tales que los dos nodos son funciones que van del dominio de los individuos al dominio de los valores de verdad, entonces [[A]] = \x. [[B]]=[[C]]=1

Las fcfg implementan la función intepretación como valor de un rasgo semántico y reemplazando las reglas que dependen de la forma del árbol directamente por restricciones en las reglas de reescritura. Estas restricciones consisten básicamente en la unificación mediante variables idénticas.

In [ ]:
nltk.data.show_cfg('gramaticas/pruebasemantica.fcfg')

In [ ]:
sents = ['Cata fuma']
grammar = 'gramaticas/pruebasemantica.fcfg'
for results in nltk.interpret_sents(sents, grammar):
    for (synrep, semrep) in results:
             print(synrep)

## Semántica eventiva

Un tipo de semántica formal que hoy ha ganado mucha popularidad es la semántica eventiva. La semántica eventiva concibe el significado de las oraciones como cuantificación sobre eventos: 

- Juan le dio ayer el libro a Pedro en la casa.
- Existe un e tal que Agente(e, Juan) & Tema(e, el libro) & Meta(e, Pedro) & en(e, la casa) & Pasado(e)

La semántica neodavidsoniana permite dispensar de grandes listas de subclases (verbos transitivos, verbos intransitivos, verbos ditransitivos, verbos bivalentes, verbos impersonales, verbos que toman distintos tipos de sintagmas preposicionales) y simplificar, en consecuencia, el léxico. Esto se hace al costo de dar una denotación más compleja a las entradas léxicas (que sin embargo, se puede automatizar si se tienen listas léxicas como las de freeling) y a las reglas de reescritura. Este tipo de enfoques propenden a sobregenerar, pero, naturalmente, este no es un problema si lo que nos interesa es parsear oraciones.

In [ ]:
sents = ['Cata dio Ficciones a Chafa']
grammar = 'gramaticas/semantica_eventiva_base.fcfg'
for results in nltk.interpret_sents(sents, grammar):
    for (synrep, semrep) in results:
             print(synrep)